In [ ]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import spacy
from tqdm import tqdm
from nltk import corpus, download as download_sw

import gensim
import gensim.corpora as corpora

In [ ]:
import json
import os
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

pattern = r'\d{7}\.json'
text_list = []
for f in os.listdir('final-rus-ds'):
  if re.search(pattern, f):
    with open(f, 'r') as json_file:
      data = json.load(json_file)
      text = data['text']
      text_list.append(text)

len(text_list)

560

In [ ]:
import spacy
import nltk
from nltk import corpus, download as download_sw

download_sw('stopwords')
stopwords = corpus.stopwords.words('russian')

nlp = spacy.load("ru2_combined_400ks_96")
nlp.add_pipe(nlp.create_pipe('sentencizer'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
texts_lemmatized = []
allowed_pos = ['NOUN', 'ADJ', 'VERB', 'ADV']

for text in tqdm(text_list):
  text = re.sub("[\:;,!?.\(\)\/\\\"\']|\d|\s|n", ' ', text)
  text = re.sub('\s{2,}', ' ', text).strip()
  doc = nlp(text)
  lemmas = []
  for token in doc:
    if token.lemma_ not in stopwords and token.pos_ in allowed_pos and len(token.lemma_) > 2:
      lemmas.append(token.lemma_)
  if len(lemmas) > 0:
    texts_lemmatized.append(lemmas)

100%|██████████| 560/560 [01:16<00:00,  7.28it/s]


In [ ]:
id2word = corpora.Dictionary(texts_lemmatized)

corp = [id2word.doc2bow(text) for text in texts_lemmatized]

In [ ]:
from gensim.models import CoherenceModel, LdaModel

gensim_coherence_values = []
gensim_model_list = []
for num_topics in tqdm(range(3, 11)):
  model = LdaModel(corpus=corp, num_topics=num_topics, id2word=id2word, passes=200, random_state=0)
  #passes – количество итераций обучения
  #random_state=0 убирает случайную генерацию тем для воспроизведения одинаковых результатов
  gensim_model_list.append(model)

  #Есть много разных метрик когерентности, самой используемой является c_v
  coherencemodel = CoherenceModel(model=model, texts=texts_lemmatized, dictionary=id2word, coherence='c_v')
  gensim_coherence_values.append(coherencemodel.get_coherence())

for i in range(len(gensim_model_list)):
  print(f'{i} {i+3} {gensim_coherence_values[i]}')

100%|██████████| 8/8 [25:40<00:00, 192.59s/it]

0 3 0.4977222729905883
1 4 0.5308540072926604
2 5 0.487235765758889
3 6 0.5316652281770679
4 7 0.5570994224388391
5 8 0.48943675126967523
6 9 0.48805858052998397
7 10 0.4864924500015654


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

gensim_vis = ldavis.prepare(gensim_model_list[4], corp, id2word)
gensim_vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.139284  0.125521       1        1  29.111028
0     -0.007338 -0.112828       2        1  17.334656
6     -0.098137  0.148362       3        1  16.668805
5      0.193464  0.027347       4        1  15.594484
2      0.185431  0.044022       5        1   8.053158
1     -0.080270 -0.198517       6        1   7.805271
3     -0.053866 -0.033907       7        1   5.432598, topic_info=          Term         Freq        Total Category  logprob  loglift
240      право   394.000000   394.000000  Default  30.0000  30.0000
303      война   640.000000   640.000000  Default  29.0000  29.0000
165    человек  2224.000000  2224.000000  Default  28.0000  28.0000
147      товар   282.000000   282.000000  Default  27.0000  27.0000
117      рынок   248.000000   248.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
223  отношение    34.731952   349.011177   Topic7  -5.3213   0.6053
73       образ    29.286686   588.795815   Topic7  -5.4918  -0.0882
2        автор    29.570492   984.488151   Topic7  -5.4822  -0.5926
127      семья    24.520376   152.645664   Topic7  -5.6694   1.0841
256    ребёнок    23.979394   267.898026   Topic7  -5.6917   0.4993

[512 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
291       2  0.013956          iii
291       4  0.530340          iii
291       5  0.446602          iii
5590      4  0.957781   абсолютизм
2         1  0.549524        автор
...     ...       ...          ...
3206      7  0.595365  юридический
3000      2  0.931655      ядерный
1343      1  0.380535         язык
1343      3  0.228321         язык
1343      7  0.390048         язык

[1176 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 7, 6, 3, 2, 4])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

text_lemma_joined = [' '.join(word_list) for word_list in texts_lemmatized]

tf_vectorizer = CountVectorizer(token_pattern = r'\b[а-яё]+\b',
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(text_lemma_joined)
print(dtm_tf.shape)

(560, 1886)


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

sklearn_model_list = []
for num_topics in tqdm(range(3, 11)):
  model = LatentDirichletAllocation(n_components=num_topics, max_iter=200, random_state=1)
  model.fit(dtm_tf)
  sklearn_model_list.append(model)

for i in range(len(sklearn_model_list)):
  print(f'{i} {i+3} {sklearn_model_list[i].score(dtm_tf)}')

100%|██████████| 8/8 [05:33<00:00, 41.68s/it]


0 3 -670670.1036759267
1 4 -668786.8444746076
2 5 -666549.6202057715
3 6 -666238.3304631944
4 7 -666852.5159478858
5 8 -666593.4608397017
6 9 -667176.9245682374
7 10 -666413.5594602925


In [ ]:
import pyLDAvis.sklearn as sklearnvis
sklearn_vis = sklearnvis.prepare(sklearn_model_list[3], dtm_tf, tf_vectorizer)
sklearn_vis

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.152332 -0.047204       1        1  30.461498
5     -0.105856 -0.222517       2        1  24.107513
4     -0.094573  0.067817       3        1  15.591990
1      0.060769  0.109477       4        1  14.097469
0     -0.203365  0.086861       5        1   9.236332
3      0.190693  0.005566       6        1   6.505198, topic_info=             Term         Freq        Total Category  logprob  loglift
1835      человек  2344.000000  2344.000000  Default  30.0000  30.0000
250   государство   624.000000   624.000000  Default  29.0000  29.0000
1129        право   459.000000   459.000000  Default  28.0000  28.0000
1343      ребёнок   281.000000   281.000000  Default  27.0000  27.0000
711         налог   269.000000   269.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
659        мнение    42.387881   278.092279   Topic6  -5.0415   0.8515
1631        также    48.095359   630.619678   Topic6  -4.9151   0.1591
179         время    38.510894   726.667194   Topic6  -5.1374  -0.2050
535      качество    35.735718   274.020185   Topic6  -5.2122   0.6955
1499  современный    34.539222   154.098803   Topic6  -5.2462   1.2371

[392 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
4         1  0.525988          автор
4         3  0.098261          автор
4         4  0.158952          автор
4         5  0.065508          автор
4         6  0.152209          автор
...     ...       ...            ...
1863      5  0.037472  экономический
1877      2  0.019040    юридический
1877      5  0.971040    юридический
1881      1  0.069180           язык
1881      6  0.925282           язык

[829 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 6, 5, 2, 1, 4])